# 다나와 사이트 자동차 실적 크롤링
## - carvarzo

In [219]:
from bs4 import BeautifulSoup
import requests

In [326]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

my_url = 'http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2018-01-00&MonthTo=' #2018년 1월 모델별 판매실적 사이트
resp = requests.get(my_url,headers=headers)
html = BeautifulSoup(resp.content,'html.parser')


In [327]:
#1월 목록만 가져오기

grid = html.find('div',{'class':'recordSection'})


In [328]:
#순위
ranks = grid.select('td.rank')

ranking = []
for rank in ranks:
    ranking.append(rank.getText())



In [329]:
#자동차 모델명
modelName = []
names = grid.findAll('input')
for name in names:
     modelName.append(name.get('title'))



In [330]:
numbers = grid.findAll('td',{'class':'right'})


산으로 가는듯한 나의 크롤링..   < td class = "right" > 의 text가 불러와지지 않아서 삽질을 했다,,

In [270]:
#판매량 크롤링

salesNum = []
rate = []    #점유율
lm = []    #전월 판매량
lm_rate = []   #전월 대비 상승/하락
ly = []    #전년도 판매량
ly_rate = []    #전년도 대비 상승/하락

for i in range(len(numbers)):
    if i%3==0:
        rate.append(numbers[i].getText())
    elif i%3==1:
        k=numbers[i].find('span').getText().replace(",","")
        lm_rate.append(k)
        lm.append(numbers[i].getText().replace(",","")[:-len(k)])
        
    else:
        y=numbers[i].find('span').getText().replace(",","")
        ly_rate.append(y)
        ly.append(numbers[i].getText().replace(",","")[:-len(y)])

### 굳이굳이 메소드로 만들어보기..

In [323]:
def danawaRecord(url):
    resp = requests.get(url,headers=headers)
    html = BeautifulSoup(resp.content,'html.parser')
    
    grid = html.find('div',{'class':'recordSection'}) #record 섹션만 가져오기
    
    #랭킹
    ranks = grid.select('td.rank')
    ranking = []
    for rank in ranks:
        ranking.append(rank.getText())
      
    #모델명
    modelName = []
    names = grid.findAll('input')
    for name in names:
        modelName.append(name.get('title'))
    
    #판매실적
    numbers = grid.findAll('td',{'class':'right'})
    salesNum = []
    rate = []    #점유율
    lm = []    #전월 판매량
    lm_rate = []   #전월 대비 상승/하락
    ly = []    #전년도 판매량
    ly_rate = []    #전년도 대비 상승/하락

    for i in range(len(numbers)):
        if i%3==0:
            rate.append(numbers[i].getText())
        elif i%3==1:
            k=numbers[i].find('span').getText().replace(",","")
            lm_rate.append(k)
            lm.append(numbers[i].getText().replace(",","")[:-len(k)])
        else:
            y=numbers[i].find('span').getText().replace(",","")
            ly_rate.append(y)
            ly.append(numbers[i].getText().replace(",","")[:-len(y)])
            
    myung = {'순위':ranking, "모델명":modelName, "점유율":rate, "전월 판매량":lm, "전월 대비 상승,하락":lm_rate, "전년도 판매량": ly, "전년도 대비 상승,하락": ly_rate}
    record = pd.DataFrame(myung)
    print(record)
    

In [324]:
danawaRecord(my_url)

      순위                모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                그랜저  5.7%   7194        468▲    9414        1752▼
1      2                포터2  4.7%   7152        905▼    7860        1613▼
2      3                쏘렌토  4.4%   6750        844▼       0        5906▲
3      4                아반떼  4.3%   6848       1171▼    5064         613▲
4      5          쏘나타 뉴 라이즈  3.9%   5708        478▼       0        5230▲
5      6             봉고3 트럭  3.7%   5195        309▼    4984          98▼
6      7                카니발  3.5%   5039        370▼    5166         497▼
7      8                 모닝  3.4%   5781       1294▼    2524        1963▲
8      9            E-class  3.2%   1544       2664▲    2669        1539▲
9     10                G80  3.0%   3092        955▲    3569         478▲
10    11           그랜드 스타렉스  2.9%      0       3903▲       0        3903▲
11    12                 코나  2.6%   2618        889▲       0        3507▲
12    13               스포티지  2.5%   44

전체 년/월 데이터 가져오기

In [325]:
final_url = "http://auto.danawa.com/auto/?Work=record&Tab=Model&Month="

for i in range(2017,2019):
    for j in range(1,13):
        if j<10:
            geturl = final_url + str(i) + "-0" + str(j) + "-00&MonthTo="
        else :
            geturl = final_url + str(i) + "-" + str(j) + "-00&MonthTo="
        print(geturl)
        danawaRecord(geturl)

http://auto.danawa.com/auto/?Work=record&Tab=Model&Month=2017-01-00&MonthTo=
      순위                   모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                   그랜저  7.7%  13833       4419▼       0        9414▲
1      2                   포터2  6.4%   9973       2113▼    8632         772▼
2      3                   쏘렌토  4.2%   7292       2101▼    7567        2376▼
3      4                   카니발  4.2%   5781        615▼    5820         654▼
4      5                   아반떼  4.1%   7799       2735▼    6996        1932▼
5      6                봉고3 트럭  4.1%   5284        300▼    4847         137▲
6      7                   스파크  3.5%   7077       2749▼    4285          43▲
7      8                   티볼리  3.1%   5613       1762▼    3222         629▲
8      9            쏘나타 (2017)  3.1%   6318       2549▼    5035        1266▼
9     10                   G80  2.9%   4243        674▼       0        3569▲
10    11                   말리부  2.9%   4149        584▼       0        3565▲

      순위                   모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                   그랜저  7.8%   9990       3122▲       0       13112▲
1      2                   포터2  5.8%   7691       2099▲   10214         424▼
2      3            쏘나타 (2017)  4.3%   4104       3136▲    5906        1334▲
3      4                   아반떼  4.2%   7353        353▼    8753        1753▼
4      5                   카니발  3.9%   5237       1277▲    6075         439▲
5      6                    모닝  3.8%   5816        527▲       0        6343▲
6      7                   쏘렌토  3.8%   5055       1273▲    7611        1283▼
7      8                   싼타페  3.2%   5997        538▼    7245        1786▼
8      9                   티볼리  3.2%   4801        623▲    4797         627▲
9     10                   SM6  2.9%   3900        948▲    6751        1903▼
10    11                  스타렉스  2.9%   3841        982▲    4370         453▲
11    12                봉고3 트럭  2.8%   5423        679▼    5753        1009▼

      순위                 모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                 그랜저  6.9%  11503        753▼       0       10750▲
1      2                 포터2  6.5%   8809       1203▲    9597         415▲
2      3                 아반떼  5.1%   8265        431▼    8472         638▼
3      4           쏘나타 뉴 라이즈  4.7%   8748       1530▼       0        7218▲
4      5                  모닝  4.2%   5456        981▲       0        6437▲
5      6                 카니발  4.0%   6302        142▼    5065        1095▲
6      7                 쏘렌토  3.6%   5343        164▲    7988        2481▼
7      8              봉고3 트럭  3.4%   5725        384▼    5122         219▲
8      9                 티볼리  3.1%   5011        287▼    5490         766▼
9     10                 싼타페  2.9%   3888        543▲    7387        2956▼
10    11            투싼(2017)  2.9%   3400       1016▲    5951        1535▼
11    12                 SM6  2.6%   3950         24▲    7901        3927▼
12    13                스

      순위                      모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량  \
0      1                      그랜저  6.7%  10194        278▼       0   
1      2                      포터2  5.9%  10064       1374▼    8276   
2      3                      아반떼  4.8%   6488        621▲    6244   
3      4                      카니발  4.2%   6573        312▼    6773   
4      5                쏘나타 뉴 라이즈  4.1%   8566       2494▼       0   
5      6                      쏘렌토  4.1%   6176        159▼    5483   
6      7                       모닝  3.6%   6724       1357▼       0   
7      8                      티볼리  3.0%   4813        334▼    4409   
8      9                   봉고3 트럭  2.9%   4597        283▼    4588   
9     10                      스파크  2.9%   3925        300▲    5726   
10    11                 투싼(2017)  2.8%   4272        162▼    3443   
11    12                      싼타페  2.5%   4443        768▼    4670   
12    13                     스타렉스  2.5%   4230        556▼    3929   
13    14            

      순위                    모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                    쏘렌토  6.5%   7768       2248▲       0       10016▲
1      2                    그랜저  5.8%   6609       2340▲       0        8949▲
2      3                    포터2  4.8%   6550        908▲    4434        3024▲
3      4                    아반떼  4.6%   7449        371▼    5135        1943▲
4      5                    카니발  3.9%   5247        764▲    3927        2084▲
5      6              쏘나타 뉴 라이즈  3.9%   5971          9▲       0        5980▲
6      7                     모닝  3.6%   5988        393▼       0        5595▲
7      8                 봉고3 트럭  3.6%   3559       1992▲    2560        2991▲
8      9                     코나  3.5%   4230       1156▲       0        5386▲
9     10                    티볼리  3.3%   4187        910▲       0        5097▲
10    11               투싼(2017)  2.9%   4132        386▲    3720         798▲
11    12                   스타렉스  2.6%   3376        569▲    2623

...

      순위                 모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                 싼타페  7.0%   1647       9972▲       0       11619▲
1      2                 포터2  5.4%   7194       1687▲    9790         909▼
2      3                 그랜저  5.2%   7359       1294▲   13112        4459▼
3      4                 쏘렌토  4.2%   5853       1112▲       0        6965▲
4      5                 아반떼  3.6%   5807        121▲    7000        1072▼
5      6                 카니발  3.4%      0       5708▲       0        5708▲
6      7              봉고3 트럭  3.3%   4412       1123▲    4744         791▲
7      8           쏘나타 뉴 라이즈  3.2%   4781        596▲       0        5377▲
8      9                  모닝  3.2%   4560        793▲    6343         990▼
9     10                  K5  2.9%   3664       1174▲       0        4838▲
10    11            그랜드 스타렉스  2.7%   3333       1207▲       0        4540▲
11    12             E-class  2.7%   2519       1975▲    3217        1277▲
12    13                 

      순위                모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                싼타페  6.6%  10996        624▼       0       10372▲
1      2                포터2  5.9%   9444        140▼   10012         708▼
2      3                카니발  5.1%   8828        826▼       0        8002▲
3      4                그랜저  5.0%   8176        261▼   10750        2835▼
4      5                아반떼  4.2%   5898        667▲    7834        1269▼
5      6                쏘렌토  3.5%   5237        322▲       0        5559▲
6      7             봉고3 트럭  3.5%   5303        252▲    5341         214▲
7      8          쏘나타 뉴 라이즈  3.3%   5365        174▼    7218        2027▼
8      9                 모닝  3.2%   5293        272▼    6437        1416▼
9     10                 K3  3.2%   5652        634▼       0        5018▲
10    11            렉스턴 스포츠  2.5%   2949        995▲       0        3944▲
11    12                티볼리  2.3%   3341        319▲       0        3660▲
12    13                 코나  2.2%   34

      순위                모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                싼타페  6.4%   9064        827▲       0        9891▲
1      2                포터2  5.2%   8442        439▼    8690         687▼
2      3                아반떼  4.9%   5928       1594▲    7109         413▲
3      4                카니발  4.8%   7059        415▲       0        7474▲
4      5                그랜저  4.3%   6674        107▼    9916        3349▼
5      6                쏘렌토  3.9%   6318        262▼       0        6056▲
6      7          쏘나타 뉴 라이즈  3.6%   4941        669▲    6072         462▼
7      8             봉고3 트럭  3.4%   5175         86▲    4314         947▲
8      9                 모닝  3.3%   4898        263▲    5367         206▼
9     10           그랜드 스타렉스  2.9%   5126        601▼       0        4525▲
10    11            렉스턴 스포츠  2.6%   4008          4▲       0        4012▲
11    12                 K5  2.5%   3443        441▲       0        3884▲
12    13                티볼리  2.4%   36

      순위                모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1                싼타페  6.5%   9803       1477▼       0        8326▲
1      2                포터2  5.4%   6157        719▲    7458         582▼
2      3                카니발  4.5%   6918       1158▼       0        5760▲
3      4                그랜저  4.4%   7600       2036▼    8949        3385▼
4      5                아반떼  4.3%   8136       2648▼    7078        1590▼
5      6           그랜드 스타렉스  3.8%   2593       2281▲       0        4874▲
6      7          쏘나타 뉴 라이즈  3.3%   5541       1394▼    5980        1833▼
7      8             봉고3 트럭  3.2%   4849        750▼    5551        1452▼
8      9                쏘렌토  3.1%   4759        816▼   10016        6073▼
9     10                 모닝  3.0%   5180       1351▼    5595        1766▼
10    11                 투싼  2.9%   4808       1112▼       0        3696▲
11    12                스파크  2.5%   3303        145▼       0        3158▲
12    13                티볼리  2.4%   37

      순위          모델명   점유율 전월 판매량 전월 대비 상승,하락 전년도 판매량 전년도 대비 상승,하락
0      1          싼타페  5.5%   9781        780▼       0        9001▲
1      2          포터2  5.5%   9261        403▼    9601         743▼
2      3          그랜저  4.7%   6737        877▲    7879         265▼
3      4          카니발  4.0%   6829        258▼       0        6571▲
4      5          아반떼  3.8%   7228        985▼    7183         940▼
5      6          쏘렌토  3.8%   5346        767▲    8107        1994▼
6      7       봉고3 트럭  3.4%   5505         39▲    5453          91▲
7      8           모닝  3.3%   5289         44▲    6010         677▼
8      9     그랜드 스타렉스  3.1%   4981        109▲       0        5090▲
9     10    쏘나타 뉴 라이즈  3.0%   5993       1084▼    6807        1898▼
10    11           K5  2.8%   3902        627▲       0        4529▲
11    12           투싼  2.6%   4856        578▼       0        4278▲
12    13          티볼리  2.6%   3910        344▲    4298          44▼
13    14      렉스턴 스포츠  2.5%   4030         72▲  